# Import all necessary packages

In [1]:
import scipy.signal as signal
import numpy as np
import matplotlib.pyplot as plt

Load npy data

In [9]:
eeg_all = np.load(r"C:\Users\admin\Documents\easy-on-the-eyes\Data\sub-P004_ses-S001_task-T1_run-001_eeg.npy")

#separate np data 

# Compute the power spectral density of each epoch


In [ ]:
# PSD settings
window_size = 4 # Length of window for PSD [sec]

# Preallocate list for PSDs
eeg_pxx = [None]*len(eeg_epochs)

for e,epoch in enumerate(eeg_epochs):
    [eeg_f, eeg_pxx[e]] = signal.welch(epoch.T, eeg_fs, nperseg=window_size*eeg_fs)

# Oganize the PSDs

In [ ]:
epoch_labels

####this is for some messed up labels in pilot data:
epoch_labels_v2 = []

for label in epoch_labels:
    if label == "tvep,1,-1,1,2Min":
        epoch_labels_v2.append("tvep,1,-1,1,2, Min")
    elif label == "tvep,1,-1,1,9.6Min":
        epoch_labels_v2.append("tvep,1,-1,1,9.6, Min")
    elif label == "tvep,1,-1,1,16Min":
        epoch_labels_v2.append("tvep,1,-1,1,16, Min")
    elif label == "tvep,1,-1,1,36Min":
        epoch_labels_v2.append("tvep,1,-1,1,36, Min")
    else:
        epoch_labels_v2.append(label)


# SORT
unique_labels = list(set(epoch_labels_v2))
types_of_stimuli = list(set([label.split(",")[-1] for label in unique_labels]))


# Remove labels not associated with a stimulus type
stimuli_to_remove = ["Stimulus Off", " eyes open", " eyes closed"]#" Static"]
for stimulus in stimuli_to_remove:
    try: { types_of_stimuli.remove(stimulus) }
    except: { print(f'Could not remove {stimulus}') }

# types_of_stimuli.remove("Off")
dict_of_stimuli = {i: v for i, v in enumerate(types_of_stimuli)}
dict_of_freqs = {0:"2", 1:"9.6", 2:"16", 3:"36"}

# Empty list to store all computed PSDs
# - Each element of the list will include PSD as [chans, freqs]
psd_stim = [[[] for j in range(len(dict_of_freqs))] for i in range(len(dict_of_stimuli))]
psd_stim_off = []

# Iterate through all the labels of stimuli and organize PSD in [stimuli][freq]
for e, epoch in enumerate(epoch_labels_v2):
    for s, stim in dict_of_stimuli.items():
        for f, freq in dict_of_freqs.items(): 
            if epoch == f"tvep,1,-1,1,{freq},{stim}":
                # ELI USE THIS VARIABLE
                psd_stim[s][f].append(eeg_pxx[e])

# Do the same but for "Stimulus Off" case
for e, epoch in enumerate(epoch_labels_v2):
    if epoch == "Stimulus Off":
        psd_stim_off.append(eeg_pxx[e])

# Create numpy arrays
psd_np_stim = [[[] for j in range(len(dict_of_freqs))] for i in range(len(dict_of_stimuli))]
psd_np_stim_off = np.array(psd_stim_off)

for s,stim in enumerate(psd_stim):
    for f,freq in enumerate(stim):
        psd_np_stim[s][f] = np.array(freq)
psd_np_stim = np.array(psd_np_stim) # Array with shape [stimulus, freq, trial, chans, samples]


# Visualize PSDs

In [ ]:
# Plot settings
f_limits = [1.5, 35]  # Frequency limits for the plots [min, max][Hz]

plot_psd = True # Enable to see plots

if plot_psd:
    for s,stim in enumerate(psd_np_stim):
        fig, ax= plt.subplots(2,2)
        fig.suptitle(dict_of_stimuli[s])
        
        for f,freq in enumerate(stim):
            fmask = (eeg_f>=f_limits[0]) & (eeg_f<=f_limits[1])
            temp_freq = eeg_f[fmask]

            temp_mean = np.mean(freq, axis=0).T[fmask,:]
            temp_sd = np.std(freq, axis=0).T[fmask,:]

            row = f // 2
            col = f % 2
            ax[row, col].plot(temp_freq, temp_mean, '-')
            ax[row, col].set_title(f"{dict_of_freqs[f]} Hz")
            ax[1,1].legend(ch_names)
            
        
        ax[0,0].set_ylabel("PXX [$\mu$V$^2$/Hz]")
        ax[1,0].set_ylabel("PXX [$\mu$V$^2$/Hz]")
        ax[1,0].set_xlabel("Frequency [Hz]")
        ax[1,1].set_xlabel("Frequency [Hz]")
        plt.tight_layout()
